<a href="https://colab.research.google.com/github/tomzw11/rental-listing-inquiries/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os

# print(os.getcwd())
df_train = pd.read_json("/content/drive/My Drive/Colab Notebooks/train.json")

# df_train.head()

In [0]:
# distribution of manager_id.
from collections import Counter

manager_id = df_train['manager_id']
Counter(list(manager_id)).most_common(20)


In [0]:
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# preprocessing.
labels = np.array(df_train["interest_level"])
le = preprocessing.LabelEncoder()
le.fit(labels)
labels_encoded = le.transform(labels)
# print(labels_encoded[0:10])
# print(labels_encoded.dtype)

# extract features to train.
features = np.array(df_train[["bathrooms","bedrooms","latitude","longitude","price"]])

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.25)

# print('Training Features Shape:', train_features.shape)
# print('Training Labels Shape:', train_labels.shape)
# print('Testing Features Shape:', test_features.shape)
# print('Testing Labels Shape:', test_labels.shape)

In [11]:
# create a baseline model based on price and number of bedrooms.
train_price = np.transpose(train_features)[4]
train_bedroom = np.transpose(train_features)[1]
# train_bedroom = [num if num>0 for num in train_bedroom]
train_baseline = [1000*b/p for (p,b) in zip(train_price,train_bedroom)]

p_33 = np.percentile(train_baseline,33)
p_66 = np.percentile(train_baseline,66)

predictions_baseline = [0 if b < p_33 else 2 if b > p_66 else 1 for b in train_baseline]

accuracy_baseline = len(predictions_baseline)
for p in zip(predictions_baseline,test_labels):
  if p[0]!=p[1]:
    accuracy_baseline = accuracy_baseline - 1

print("Baseline Prediction Accuracy ",accuracy_baseline/len(predictions_baseline))


Baseline Prediction Accuracy  0.777354514508024


In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_features,train_labels)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
predictions = clf.predict(test_features)

accuracy = len(predictions)
for p in zip(predictions,test_labels):
  if p[0]!=p[1]:
    accuracy = accuracy - 1

print("Prediction Accuracy ",accuracy/len(predictions))


Prediction Accuracy  0.7032744366996272


In [0]:
# visualize decision tree.

# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = clf.estimators_[5]

feature_list = ["bathrooms","bedrooms","latitude","longitude","price"]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')


In [17]:
# figure out variable importances.

# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: price                Importance: 0.35
Variable: latitude             Importance: 0.29
Variable: longitude            Importance: 0.29
Variable: bedrooms             Importance: 0.05
Variable: bathrooms            Importance: 0.02
